In [2]:


import numpy as np
import pandas as pd
import xarray as xr

from earthkit.data import from_object, from_source
from earthkit.data.readers import Reader
from earthkit.data.utils import module_inputs_wrapper

import dummy_module
from dummy_module import XR_TYPES

In [3]:

TEST_NP = np.arange(10)
TEST_NP2 = np.arange(10)

TEST_DF = pd.DataFrame({"index": TEST_NP, "data": TEST_NP2}).set_index("index")

TEST_DA = xr.DataArray(TEST_NP, name="test")
TEST_DA2 = xr.DataArray(TEST_NP2, name="test2")
TEST_DS = TEST_DA.to_dataset()
TEST_DS["test2"] = TEST_DA2

EK_GRIB_READER = from_source("file", "../data/test_single.grib")
EK_XARRAY_WRAPPER = from_object(TEST_DS)
EK_NUMPY_WRAPPER = from_object(TEST_NP)

WRAPPED_XR_ONES_LIKE = module_inputs_wrapper.transform_function_inputs(
    xr.ones_like, kwarg_types={"other": XR_TYPES}
)

WRAPPED_XR_ONES_LIKE_TYPE_SETTING = module_inputs_wrapper.transform_function_inputs(
    dummy_module.xarray_ones_like,
)

WRAPPED_NP_MEAN = module_inputs_wrapper.transform_function_inputs(
    np.mean,
    kwarg_types={"a": np.ndarray},
    convert_types=(
        Reader
    ),  # Only convert Earthkit.data.Reader (np.mean can handle xarray and pandas)
)

# WRAPPED_NP_MEAN_TYPE_SETTING = module_inputs_wrapper.transform_function_inputs(
#     dummy_module.numpy_mean,
# )

WRAPPED_DUMMY_MODULE = module_inputs_wrapper.transform_module_inputs(
    dummy_module,
)



In [11]:
# np.mean(EK_GRIB_READER)
result = WRAPPED_NP_MEAN_TYPE_SETTING(EK_GRIB_READER)
assert result  == np.mean(EK_GRIB_READER.to_numpy())
assert isinstance(result, np.float64)


In [6]:

# Test with Earthkit.data XarrayWrapper object
ek_object_result = WRAPPED_DUMMY_MODULE.numpy_mean(EK_XARRAY_WRAPPER)
assert ek_object_result == np.mean(TEST_DS)
# assert isinstance(ek_object_result, type(EK_XARRAY_WRAPPER.data))

type(ek_object_result)

xarray.core.dataset.Dataset

In [ ]:

# Test with Earthkit.data NumpyWrapper object
ek_object_result = WRAPPED_DUMMY_MODULE.numpy_mean(EK_NUMPY_WRAPPER)
assert ek_object_result == np.mean(TEST_NP)
assert isinstance(ek_object_result, np.float64)